In [62]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats 

import networkx as nx

from mesa import Model, Agent
from mesa.time import RandomActivation
from mesa.space import NetworkGrid
from mesa.datacollection import DataCollector

import time

Our scenario shows a population of $n$ individuals, organized in a network whose edges denote close relationships. Outside the population, an actor, which will be referred to as "influencer", is interested in having an idea $I$ be accepted by as many agents as possible, and has some means to influence the spread of such idea in the network. 

Each agent $i$ is characterized by an "opinion" attribute $p_i$, representing the personal position with respect to the idea. $p_i \in [0, 1]$, where 0 represents "full disagreement", 0.5 "neutrality", and 1 "full agreement". 

Our model of choice is the beta distribution: $p_i(0) \sim \text{Beta}(\alpha, \beta)$. \
The parameters of the distributions are set as $\alpha = s \cdot \mu$, $\beta = s \cdot (1 - \mu)$, where $\mu = E[p_i(0)]$; thus, $1/\mu$ is a measure of how unorthodox and clashing is $I$ for the population, and $1/s$ is a measure of the polarizing nature of $I$. This intepretation of $s$ derives from noticing its relation with the variance $$V(p_i(0)) = \frac{\mu(1-\mu)}{s+1}$$ and its relation with the skewness of the distribution $$\text{sk}(p_i(0)) = \frac{(2 - 4\mu)\sqrt{s + 1}}{\sqrt{\mu(1 - \mu)}(s + 2)}$$ implying that $\frac{\text{d}|\text{sk}(p_i(0))|}{\text{d}s} < 0$. By any means, we can represent the idea as a vector of these two parameters $I = (1/\mu, 1/s)$. We incorporate in the model a mechanism by which, when the network is active, each node has a probability of removing the link with then neighbor with the most distant opinion, provided that one of them disagrees and one of them agrees, that is $\text{sign}(0.5 - p_j) \neq \text{sign}(0.5 - p_i)$.

After instantiating the network, generated from the Albert-Barabàsi model, we assign the opinion attribute to the nodes sampling from our chosen distribution. Once $p_i$ have been assigned, we can compute for each node the value of an "exposition" attribute, defined as the average opinion of neighbors, weighted by their similarity $e_i = \frac{1}{k_i - \sum_j(|p_j - p_i|)}\sum_{j \in N(i)}(1 - |p_j - p_i|)p_j$.

The influencer controls a model variable denoted as $M(t)$, representing the degree of public discussion/media coverage on the idea; we thus assume that the external actor is able to influence public awareness and attention on $I$, but not to directly convince agents, whose opinions only directly change due to their interactions. $M$ goes from 0, no awareness or discussion, to 1, full, bombarding propaganda. Initially, $M(0) = 0$, and the network is inactive: the opinions attributes represent private or even unconscious belief about $I$, but have no way to induce any interaction until people start discussing the topic. Once the influencer increases $M$, the network becomes active. 
At each time step $t$, the following happens: 
1. $M(t)$ is set, through some strategy.
2. If $M(t) > 0$, for each node $i$ get its neighbor $j$ such that $p_j = \text{arg max}_{p_j} |p_j - p_i|$. If $\text{sign}(0.5 - p_j) \neq \text{sign}(0.5 - p_i)$, then remove the $i, j$ link with probability $q = M(t)|p_j - p_i|$. 
3. The exposition attributes for each node $e_i(t)$ are computed on the basis of $p_j(t-1)$.
4. The opinions attributes are updated according to: $$p_i(t) = (1 - M(t))p_i(t-1) + M(t)e_i(t)$$
 


The parameters we are going to vary to look at variations in model behaviour are: the type, size and order of the network, $\mu$, $s$ and the strategy $M$.\
The attributes we are interested in storing at each step and for each parameter combination are: the distribution of opinions $P(p)$, from which mean and variance can be calculated, the distribution of expositions $P(e)$, the degree of assortativity of the graph (nodes with $p$ above or below 0.5) and the average degree of the graph.  

In [52]:
def get_assortativity(model):
    return nx.attribute_assortativity_coefficient(model.G, attribute = "agrees")
    
def get_avgk(model):
    return np.mean([k for n,k in model.G.degree])

def get_remlinks(model):
    return model.removed_links


# def get_avgd(model):
#     return nx.average_shortest_path_length(model.G)

# def get_diam(model):
#     return nx.diameter(model.G)

In [66]:
class iModel(Model):
    
    def __init__(self, G, mu, s, strategy, seed = None):
        
        self.G = G.copy()
        self.n = len(self.G.nodes())
        self.mu = mu
        self.s = s
        self.strategy = strategy
        self.removed_links = []
        self.grid = NetworkGrid(self.G)
        self.schedule = RandomActivation(self)
        
        np.random.seed(seed)
        pi = np.random.beta(self.s*self.mu, self.s*(1-self.mu), self.n)
        st = np.sign(pi - 0.5)
        nx.set_node_attributes(G, values = {i:st[i].item() for i in range(self.n)}, name = "agrees")
        
        for i, node in enumerate(self.G.nodes()):
            a = iAgent(node, self, p = pi[i], status = st[i])
            self.schedule.add(a)
            self.grid.place_agent(a, node)
        
        self.datacollector = DataCollector(
            
            model_reporters = {"Assortativity": get_assortativity,
                               "Average degree": get_avgk,
                               "links removed": get_remlinks},
            
            agent_reporters = {"Opinion": "p", "Exposition": "e"}
        )
        
        self.running = True
        self.datacollector.collect(self)
    def step(self):

        self.M = next(self.strategy)
        self.schedule.step()
        self.datacollector.collect(self)


class iAgent(Agent):
    
    def __init__(self, unique_id, model, p, status):
        super().__init__(unique_id, model)
        self.p = p
        self.e = 0
        self.status = status
    
    def link_removal(self):
        neighbors = self.model.grid.get_neighbors(self.pos, include_center = False)
        if len(neighbors) > 0:
            d, a_d = 0, None
            for a in neighbors:
                if abs(self.p - a.p) > d and np.sign(self.p - 0.5) != np.sign(a.p - 0.5):
                    d = abs(self.p - a.p)
                    a_d = a
            if d != 0:
                if self.random.random() < self.model.M * d:
                    self.model.G.remove_edge(self.unique_id, a_d.unique_id)
                    self.model.removed_links.append((self.unique_id, a_d.unique_id))
                    
    
    def update_e(self):
        neighbors = self.model.grid.get_neighbors(self.pos, include_center = False)
        if len(neighbors) > 0:
            sum_e = 0
            sum_weight = 0
            for a in neighbors:
                w = (1 - abs(self.p - a.p))
                sum_e += w*a.p
                sum_weight += w
            self.e = sum_e/sum_weight
            if self.e is np.nan:
                self.e = 0
    
    def update_p(self):
        neighbors = self.model.grid.get_neighbors(self.pos, include_center = False)
        if len(neighbors) > 0:
            self.p = (1 - self.model.M)*self.p + self.model.M*self.e
    
    def step(self):
        if self.model.M > 0:
            #print("link_removal: START")
            #t = time.time()
            self.link_removal()
            #print("link_removal: time:", time.time() - t)

            self.update_e()
            self.update_p()


In [284]:
class iModel_list(iModel):
    
    def __init__(self, G, mu, s, strategy, seed = None):
        super().__init__(G, mu, s, strategy, seed)
        self.count = 0
        
    
    def step(self):
        
        self.M = self.strategy[self.count]
        self.count += 1
        self.schedule.step()
        self.datacollector.collect(self)

The following version of the model modifies the influence by making it present only for nodes with opinion above/below a certain quantile.

In [84]:
class iModel2(Model):
    
    def __init__(self, G, mu, s, strategy, quantile = 0.2, contrast = True, seed = None):
        
        self.G = G.copy()
        self.n = len(self.G.nodes())
        self.mu = mu
        self.s = s
        self.strategy = strategy
        self.quantile = quantile
        self.contrast = contrast
        self.removed_links = []
        self.grid = NetworkGrid(self.G)
        self.schedule = RandomActivation(self)
        
        np.random.seed(seed)
        pi = np.random.beta(self.s*self.mu, self.s*(1-self.mu), self.n)
        st = np.sign(pi - 0.5)
        nx.set_node_attributes(G, values = {i:st[i].item() for i in range(self.n)}, name = "agrees")
        
        for i, node in enumerate(self.G.nodes()):
            a = iAgent2(node, self, p = pi[i], status = st[i])
            self.schedule.add(a)
            self.grid.place_agent(a, node)
        
        self.datacollector = DataCollector(
            
            model_reporters = {"Assortativity": get_assortativity,
                               "Average degree": get_avgk,
                               "links removed": get_remlinks},
            
            agent_reporters = {"Opinion": "p", "Exposition": "e"}
        )
        
        self.running = True
        self.datacollector.collect(self)
        
    def step(self):

        self.M = next(self.strategy)
        self.schedule.step()
        self.datacollector.collect(self)


class iAgent2(Agent):
    
    def __init__(self, unique_id, model, p, status):
        super().__init__(unique_id, model)
        self.p = p
        self.e = 0
        self.status = status
        
    def check_quantile(self):

        if (self.model.contrast and self.model.mu < 0.5) or (not self.model.contrast and self.model.mu >= 0.5):            
            return self.p >= np.quantile([a.p for a in self.model.schedule.agents], self.model.quantile)
        else:  
            return self.p <= np.quantile([a.p for a in self.model.schedule.agents], self.model.quantile)
    
    def link_removal(self):
        neighbors = self.model.grid.get_neighbors(self.pos, include_center = False)
        if len(neighbors) > 0:
            d, a_d = 0, None
            for a in neighbors:
                if abs(self.p - a.p) > d and np.sign(self.p - 0.5) != np.sign(a.p - 0.5):
                    d = abs(self.p - a.p)
                    a_d = a
            if d != 0:
                if self.random.random() < self.model.M * d:
                    self.model.G.remove_edge(self.unique_id, a_d.unique_id)
                    self.model.removed_links.append((self.unique_id, a_d.unique_id))
                    
    
    def update_e(self):
        neighbors = self.model.grid.get_neighbors(self.pos, include_center = False)
        if len(neighbors) > 0:
            sum_e = 0
            sum_weight = 0
            for a in neighbors:
                w = (1 - abs(self.p - a.p))
                sum_e += w*a.p
                sum_weight += w
            self.e = sum_e/sum_weight
            if self.e is np.nan:
                self.e = 0
    
    def update_p(self):
        neighbors = self.model.grid.get_neighbors(self.pos, include_center = False)
        if len(neighbors) > 0:
            self.p = (1 - self.model.M)*self.p + self.model.M*self.e
    
    def step(self):
        if self.model.M > 0:
            if self.check_quantile():
                self.link_removal()
                self.update_e()
                self.update_p()


In [83]:
class iModel2_list(iModel2):
    
    def __init__(self, G, mu, s, strategy, quantile = 0.2, contrast = True, seed = None):
        super().__init__(G, mu, s, strategy, quantile, contrast, seed)
        self.count = 0
        
    
    def step(self):
        
        self.M = self.strategy[self.count]
        self.count += 1
        self.schedule.step()
        self.datacollector.collect(self)